In [1]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.segmentation import watershed
from skimage.morphology import disk, dilation, erosion
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherin_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-A-2/Cadherins'
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-2/Nuclei'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-2/Cell'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files
cadherin_files = glob.glob(os.path.join(cadherin_dir, '*Cadherins.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Process each file
for cadherin_file in cadherin_files:
    # Get base filename for finding corresponding nuclei mask
    filename = os.path.basename(cadherin_file)
    base_name = filename.replace('_Cadherins.tif', '')
    nuclei_mask_file = os.path.join(nuclei_dir, f"{base_name}_Nuclei_mask.tif")

    # Check if nuclei mask exists
    if not os.path.exists(nuclei_mask_file):
        print(f"WARNING: Nuclei mask not found for {filename}, skipping")
        continue

    print(f"Processing: {filename}")

    # Load the images
    cadherin_img = tifffile.imread(cadherin_file)
    nuclei_masks = tifffile.imread(nuclei_mask_file)

    print(f"  Cadherin image shape: {cadherin_img.shape}")
    print(f"  Nuclei mask shape: {nuclei_masks.shape}")

    # Extract Cadherin channel if needed
    if len(cadherin_img.shape) == 2:
        # Single channel image (already Cadherin)
        print("  Detected single-channel Cadherin image")
        membrane_channel = cadherin_img
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        membrane_channel = cadherin_img[0]  # First channel
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        membrane_channel = cadherin_img[:, :, 0]  # First channel
    else:
        print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
        if len(cadherin_img.shape) == 3:
            membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
        else:
            membrane_channel = cadherin_img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    membrane_smoothed = gaussian(membrane_channel, sigma=1)

    # Calculate morphological gradient
    print("  Calculating morphological gradient...")
    selem = disk(1)  # Adjust radius if needed
    dilated = dilation(membrane_smoothed, selem)
    eroded = erosion(membrane_smoothed, selem)
    membrane_gradient = dilated - eroded

    # Normalize gradient to 0-1 range
    membrane_gradient_norm = (membrane_gradient - membrane_gradient.min()) / (
                membrane_gradient.max() - membrane_gradient.min())

    # Apply watershed segmentation using nuclei as seeds
    print("  Performing watershed segmentation...")
    watershed_output = watershed(membrane_gradient_norm, nuclei_masks, mask=membrane_gradient_norm > 0)

    # Save only the cell mask result
    cell_mask_path = os.path.join(output_dir, f"{base_name}_cell_mask.tif")
    tifffile.imwrite(cell_mask_path, watershed_output.astype(np.uint32))
    print(f"  Saved cell mask to {cell_mask_path}")

    # Print statistics
    print(f"  Number of nuclei (seeds): {len(np.unique(nuclei_masks)) - 1}")
    print(f"  Number of segmented cells: {len(np.unique(watershed_output)) - 1}")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Mounted at /content/drive
Found 18 Cadherin files to process
Processing: denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_Cadherins.tif
  Cadherin image shape: (1024, 1024)
  Nuclei mask shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Calculating morphological gradient...
  Performing watershed segmentation...
  Saved cell mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-2/Cell/denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_cell_mask.tif
  Number of nuclei (seeds): 241
  Number of segmented cells: 241
  Processing complete for denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq001_contrast_Cadherins.tif

Processing: denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq002_contrast_Cadherins.tif
  Cadherin image shape: (1024, 1024)
  Nuclei mask shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Calculating morphological gradient...
  Performing watershed segmentation...
